## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb　←いまここ
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

In [ ]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import re

## 研究者を抽出する関数

研究者情報は、grantAward/summary/memberとgrantAward/memberList/memberの2箇所にある。前者は同じ人は複数出てこなくてまとまっているが、所属機関等のコードがない。後者は所属機関コードがあるが、毎年度の実績報告書があるので同じ人が複数回出てくる。差し当たって前者からデータを取得することにする。そのうち余裕が出たら、後者のデータと突合したい。

In [ ]:
def member(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    memberlist = []
    for grantAward in tree.iterfind("grantAward"):
        awardnumber = grantAward.get("awardNumber")
        for member in grantAward.find("summary[@xml:lang='ja']", nsmap).iterfind("member", nsmap):
            sequence = member.get("sequence")
            role = member.get("role")
            kenkyuusha_id = member.get("eradCode")

            try:
                familyname = member.find("personalName/familyName").text
            except:
                familyname = np.NaN

            try:
                givenname = member.find("personalName/givenName").text
            except:
                givenname = np.NaN

            row = [
                awardnumber,
                sequence,
                role,
                kenkyuusha_id,
                familyname,
                givenname,
            ]
            memberlist.append(row)

    df = pd.DataFrame(memberlist)
    df.columns = [
        'awardnumber',
        'sequence',
        'role',
        'kenkyuusha_id',
        'familyname',
        'givenname',
    ]

    pickledfile = 'pickledDF_member_from_summary/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
    df.to_pickle(pickledfile)

フォルダをいったんきれいにする関数を定義して、実行する

In [ ]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_member_from_summary')

関数を実行して、XMLファイルごとにパースして、pickleして保存する

In [ ]:
for xmlfile in tqdm(glob.glob('xml/201*.xml')):
    member(xmlfile)

In [ ]:
columns = [
    'awardnumber',
    'sequence',
    'role',
    'kenkyuusha_id',
    'familyname',
    'givenname',
]
df = pd.DataFrame(columns=columns)

In [ ]:
for dump in tqdm(glob.glob('pickledDF_member_from_summary/*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

連結したデータフレームをいったんpickleして保存

In [ ]:
df.to_pickle('beforeCleaning/parse_member_from_summary.dump')

### データクリーニング

pickleからデータフレームを復元する

In [ ]:
with open('beforeCleaning/parse_member_from_summary.dump', mode='rb') as f:
    df = pickle.load(f)

データフレームの構造を概観する

In [ ]:
df.info()

In [ ]:
df = df.fillna(0)
df.info()

研究者番号のルールに合わないもの（数字以外の文字が含まれているもの）を抽出する

まず、インデックスをリセットしておく。

In [ ]:
df = df.reset_index(drop=True)
df

In [ ]:
df['kenkyuusha_id'].str.match('^[0-9]*$').value_counts()

Falseが2件あった。Falseのindexを抽出して、データフレームで表示する

In [ ]:
falselist = df['kenkyuusha_id'].str.match('^[0-9]*$')
falselist = list(falselist[falselist == False].index)
df.loc[falselist]

KAKENやresearchmapで個人名で検索して正当な研究者番号に置換する

In [ ]:
#df.kenkyuusha_id.replace('235000 6', 50004619, inplace=True)
#df.kenkyuusha_id.replace('2033+220', 80224103, inplace=True)
#df.kenkyuusha_id.replace('A9406506', 10226110, inplace=True)
df.kenkyuusha_id.replace('08J05773', 50571535, inplace=True)
df.kenkyuusha_id.replace('12J00079', 40737251, inplace=True)

In [ ]:
df.loc[falselist]

In [ ]:
df = df.astype({
    'sequence': np.int64,
    'kenkyuusha_id': np.int64,
})
df.info()

重複があるか？

In [ ]:
df.duplicated().any()

role の件数を確認しておく

In [ ]:
df.role.value_counts()

roleの対訳

|role|対訳|
|:---|:---|
|principal_investigator|研究代表者|
|co_investigator_buntan|研究分担者|
|co_investigator_renkei|連携研究者|
|research_collaborator|研究協力者|
|research_fellow|特別研究員奨励費 特別研究員|
|foreign_research_fellow|特別研究員奨励費 外国人特別研究員|
|host_researcher|特別研究員奨励費 受入研究者|
|area_organizer|特定領域研究、新学術領域研究(研究領域提案型) 領域代表者|
|co_investigator_buntan_support|新学術領域研究（研究領域提案型）『学術研究支援基盤形成』 研究支援分担者|
|principal_investigator_support|新学術領域研究（研究領域提案型）『学術研究支援基盤形成』 研究支援代表者|

### roleから次のものだけ抽出

- 研究代表者、
- 特定領域研究、新学術領域研究(研究領域提案型) 領域代表者、
- 新学術領域研究（研究領域提案型）『学術研究支援基盤形成』 研究支援代表者

In [ ]:
df = df[(df['role'] == 'principal_investigator') | (df['role'] == 'area_organizer') | (df['role'] == 'principal_investigator_support')]
df

awardnumberごとにsequenceが最大のレコードのみ抽出する。生のXMLを眺めてみると、sequenceが大きいほど古い年度のデータなので。

In [ ]:
seqmax = df.groupby('awardnumber')['sequence'].max().reset_index()
seqmax

awardnumberがユニークかどうか確認するために、行数234177とawardnumberのユニークな値の数が一致するかどうか確認する。

In [ ]:
seqmax['awardnumber'].nunique(dropna=False)

awardnumberがユニークになった。

In [ ]:
df = pd.merge(seqmax, df, on=['awardnumber', 'sequence'])
df

In [ ]:
seqmax['awardnumber'].nunique(dropna=False)

awardnumberはユニークだったので（ユニークじゃなかったら何かが違う）、インデックスにする。

In [ ]:
df = df.set_index('awardnumber')
df

In [ ]:
df.to_pickle('afterCleaning/parse_member_from_summary.dump')

ひとまず終了。

次は、parse_grantaward_integration.ipynbで3つの部品を統合する。